# Radar Test Data and How We Can Use it
## Marci Detwiller
### University of Saskatchewan

#### October 22, 2018

## What kind of data do we collect on site?
## What can we do with this data after its collected?
## How can we improve our radars?

Need more pictures!

Ultimately, we want to use the data we collect to improve our 
radar data quality, whether that is increasing amount of data collected,
calibrating so that elevation data is meaningful, etc.

## Data collection

### Data collected depends on instruments available
- Vector network analyzer (VNA)
- Oscilloscope ('scope')
- Multimeters
- Wattmeters

### Also depends on accessories, including specialized probes and tools
- High voltage probe
- Infrared camera
- Heat gun
- Thermistor probe for multimeter

# Some examples of data collected:

- Propagation delay measurements (VNA)
- Standing wave ratio (VNA)
- Sky noise spectrum (VNA)
- Phase output of direct digital synthesis system (scope)
- Transmitter power output (scope)
- Voltages on transmitters, settings (multimeters) 

<img align="left" style="transform:rotate(90deg);" width="800" height="400" src="./pictures/20180811_170504.jpg">

<img align="left" width="600" height="200" src="./pictures/IMG_20180810_100312193.jpg">

## Data analysis

## Some data is easily interpreted and allows us to immediately diagnose:
- Bad connections
- Faulty components
- Calibrations required

## What else can we do with this data?
- Some issues are less obvious, so additional analysis can give more insight.

# Vector Network Analyzer

<img src="./pictures/network-analyzer.jpg" width="500" align="center">

# VNA Measurements (data I will show today)

- Propagation delay measurements
    * pm-path: A transmission measurement (S12) through the phasing matrix. 
    * transmitter-path: A transmission measurement (S12) through the transmitter.
- Standing wave ratio
    * feedline-VSWR: A reflection measurement (S11) from the feedline in the building, out to the antenna, and back.
- Sky noise spectrum
    * I won't show any of this today.

In [2]:
import math
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
pd.options.mode.chained_assignment = None
import json
import sys
import random
sys.path.append('../tdiff_path/')

# import some modules that I created to do some data processing.

import dataset_operations.dataset_operations as do
import retrieve_data.retrieve_data as retrieve

In [3]:
# I have metadata for all the datasets I have available stored in a csv.
# Includes path to files and a mapping of file to channel name. 
site_file_metadata=pd.read_csv('site_file_metadata.csv')

In [4]:
# Let's check out the data we have available!
available_data = site_file_metadata.loc[:,['site','date','data_type']].sort_values(by=['site'])

print(available_data)

   site      date         data_type
27  CLY  20180817           pm-path
15  CLY  20160701           pm-path
26  CLY  20180817           pm-path
23  CLY  20180817     feedline-VSWR
10  CLY  20160701  transmitter-path
25  CLY  20180817  transmitter-path
24  CLY  20180817     feedline-VSWR
4   CLY  20160701     feedline-VSWR
3   CLY  20160628     feedline-VSWR
5   INV  20170811     feedline-VSWR
11  INV  20170813  transmitter-path
16  INV  20170813           pm-path
1   PGR  20171003     feedline-VSWR
9   PGR  20170930  transmitter-path
2   PGR  20170930     feedline-VSWR
14  PGR  20171001           pm-path
22  RKN  20180808           pm-path
21  RKN  20180808           pm-path
20  RKN  20180808  transmitter-path
7   RKN  20160721     feedline-VSWR
18  RKN  20180808     feedline-VSWR
17  RKN  20160719           pm-path
6   RKN  20160705     feedline-VSWR
19  RKN  20180808     feedline-VSWR
12  RKN  20160719  transmitter-path
0   SAS  20170627     feedline-VSWR
8   SAS  20170630  transmitt

In [5]:
# Now we can choose a site, date, and data type that we want to look at. 

%run load_data.py CLY 20180817 pm-path

# load_data.py makes the following available to us:
* working_dataframe, with all information on all channels from this site, date, and data type
* linear_fit_dict, with information on the phase path for each channel
* cable_loss_dataset_dict, if we chose feedline-VSWR datatype, which is used to calculate a single-direction path from a VSWR measurement.


In [6]:
# Here is a few lines from the working dataframe
print(working_dataframe.head())

      freq  M0magnitude  M0phase_deg  M1magnitude  M1phase_deg  M2magnitude  \
0  8000000    14.390086   144.944183    14.431127   143.980942    14.600711   
1  8030000    14.378378   139.116516    14.409729   138.240814    14.586692   
2  8060000    14.332479   133.010605    14.365749   132.192123    14.565275   
3  8090000    14.324810   126.923569    14.331370   126.037354    14.550996   
4  8120000    14.292202   121.152634    14.312615   120.305115    14.536831   

   M2phase_deg  M3magnitude  M3phase_deg  M4magnitude       ...         \
0   139.236679    14.513210   144.256424   -34.177628       ...          
1   133.397079    14.509064   138.349976   -41.843716       ...          
2   127.306061    14.476974   132.230484   -34.120739       ...          
3   121.336678    14.467180   126.256256   -48.119297       ...          
4   115.503510    14.437731   120.461250   -32.504646       ...          

   I2phase_rad  I3phase_rad  M_all_phase_deg_unwrap  M_all_phase_rad  \
0     1.

In [7]:
# We can also take a look at what's in the linear_fit_dict:
# Let's look at the dictionary that is in the linear_fit_dict['M0'], showing just the head of the DataFrames
# So we can get an overview of what's there.
viewing_dict = {key: (value.head() if type(value) == pd.Series or type(value) == pd.DataFrame else value) for (key, value) in linear_fit_dict['M0'].items()}
print(viewing_dict)

{'slope': -2.3117678830078856e-06, 'intercept': 20.20239065210879, 'rvalue': -0.9994042468829112, 'pvalue': 0.0, 'stderr': 3.996686229919998e-09, 'offset_of_best_fit_rads': 0    0.821506
1    0.789147
2    0.751931
3    0.715046
4    0.683677
Name: phase_rad, dtype: float64, 'time_delay_ns': 367.93, 'best_fit_line_rads':    phase_rad
0   1.708248
1   1.638895
2   1.569542
3   1.500188
4   1.430835}


# Time to plot some things!

## Things that I think are useful to plot:
- path phases, to see differences between individual channels.
- combined array phases, to see differences between arrays.
- magnitudes, to see if any paths have lower signal amplitude.

- Active components and differences in antennas yield a non-linear phase path across frequency which directly affects elevation measurements.

# Conclusion
- We can gain a better understanding of our data by understanding our hardware (and operating software).
- If anyone is interested in looking at or using hardware data, **talk to me!**
- If anyone has suggestions for hardware measurements that would help with data analysis, **talk to me!**

# Sources
http://www.rfwireless-world.com/Terminology/SNA-Scalar-Network-Analyzer-vs-VNA-Vector-Network-Analyzer.html

In [ ]:
# If anyone wants to take a look at what the code is doing. 
#%pycat load_data.py
#%pycat ../tdiff_path/retrieve_data/retrieve_data.py
#%pycat ../tdiff_path/dataset_operations/dataset_operations.py